In [ ]:
"""
class MyClass(MySuperClass, MyMixin):
    x = 42

    def x2(self):
        return self.x * 2

MyClass = type('MyClass',
              (MySuperClass, MyMixin),
              {'x': 42, 'x2': lambda self: self.x * 2},
          )
"""

In [2]:
"""
The type class is a metaclass: a class that builds classes. In other words, instances of
the type class are classes. 
"""
type(7)

int

In [3]:
type(int)

type

In [4]:
type(OSError)

type

In [5]:
class Whatever:
    pass

type(Whatever)

type

In [6]:
class Dog:
    def __init__(self, name, weight, owner):
        self.name = name
        self.weight = weight
        self.owner = owner

oreo = Dog('Oreo', 20, 'Mathias')
oreo

In [18]:
from typing import Union, Any
from collections.abc import Iterable, Iterator

FieldNames = Union[str, Iterable[str]]

# NOTE the return type, the return class will be a subclass
def record_factory(cls_name: str, field_names: FieldNames) -> type[tuple]:
    slots = parse_identifiers(field_names)

    def __init__(self, *args, **kwargs) -> None:
        attrs = dict(zip(self.__slots__, args))
        attrs.update(kwargs)
        for name, value in attrs.items():
            setattr(self, name, value)

    def __iter__(self) -> Iterator[Any]:
        for name in self.__slots__:
            yield getattr(self, name)

    def __repr__(self):
        values = ', '.join(f'{name}={value!r}'
                           for name, value in zip(self.__slots__, self))
        cls_name = self.__class__.__name__
        return f'{cls_name}({values})'

    cls_attrs = dict(
        __slots__=slots,
        __init__=__init__,
        __iter__=__iter__,
        __repr__=__repr__,
    )

    return type(cls_name, (object,), cls_attrs)

def parse_identifiers(names: FieldNames) -> tuple[str, ...]:
    if isinstance(names, str):
        names = names.replace(',', ' ').split()
    if not all(s.isidentifier() for s in names):
        raise ValueError('names must all be valid Identifiers')
    return tuple(names)

In [19]:
Dog = record_factory('Dog', 'name weight owner')
oreo = Dog('Oreo', 20, 'Mathias')
oreo

Dog(name='Oreo', weight=20, owner='Mathias')

In [20]:
name, weight, _ = oreo
name, weight

('Oreo', 20)

In [23]:
"{2}' dog weighs {1}kg".format(*oreo)

"Mathias' dog weighs 20kg"

In [24]:
oreo.weight = 25
oreo

Dog(name='Oreo', weight=25, owner='Mathias')

In [25]:
Dog.__mro__

(__main__.Dog, object)

In [26]:
int(), float(), bool(), str(), list(), dict(), set()

(0, 0.0, False, '', [], {}, set())

In [28]:
from collections.abc import Callable
from typing import Any, NoReturn, get_type_hints

class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.constructor = constructor

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...: # ... determines if the value is None vs values that were not given
            value = self.constructor()
        else:
            try:
                type_name = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        instance.__dict__[self.name] = value

In [42]:
import inspect

# some methods are prepended with `_` to reduce the chance of name clashes with user-defined field names
class Checked:
    @classmethod
    def _fields(cls) -> dict[str, type]:
        #return inspect.get_annotations
        return get_type_hints(cls)

    def __init_subclass__(subclass) -> None:
        #super().__init_subclass_()
        for name, constructor in subclass._fields().items():
            setattr(subclass, name, Field(name, constructor))

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            self.__flag_unknown_attrs(*kwargs)

    def __setattr__(self, name: str, value: Any) -> None:
        if name in self._fields():
            cls = self.__class__
            descriptor = getattr(cls, name)
            descriptor.__set__(self, value)
        else:
            self.__flag_unknown_attrs(name)

    def __flag_unknown_attrs(self, *name: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(
            f'{key}={value!r}' for key, value in self._asdict().items()
        )
        return f'{self.__class__.__name__}({kwargs})'

In [50]:
class Movie(Checked):
    title: str
    year: int
    box_office: float

movie = Movie(title='The Matrix', year=1999, box_office=234)
movie.title

'The Matrix'

In [51]:
movie

Movie(title='The Matrix', year=1999, box_office=234)

In [55]:
def checked(cls: type) -> type:
    for name, constructor in _fields(cls).items():
        setattr(cls, name, Field(name, constructor))
    cls._fields = classmethod(_fields)

    instance_methods = (
        __init__,
        __repr__,
        __setattr__,
        _asdict,
        __flag_unknown_attrs,
    )
    for method in instance_methods:
        setattr(cls, method.__name__, method)
    return cls

In [56]:
@checked
class Movie:
    title: str
    year: int
    box_office: float

movie = Movie(title='The Matrix', year=1999, box_office=234)
movie.title

'The Matrix'

In [57]:
# same as the class methods but this time were utilizing class @decorator noted above this cell
def _fields(cls) -> dict[str, type]:
    return get_type_hints(cls)

def __init__(self, **kwargs: Any) -> None:
    for name in self._fields():
        value = kwargs.pop(name, ...)
        setattr(self, name, value)
    if kwargs:
        self.__flag_unknown_attrs(*kwargs)

def __setattr__(self, name: str, value: Any) -> None:
    if name in self._fields():
        cls = self.__class__
        descriptor = getattr(cls, name)
        descriptor.__set__(self, value)
    else:
        self.__flag_unknown_attrs(name)

def __flag_unknown_attrs(self, *name: str) -> NoReturn:
    plural = 's' if len(names) > 1 else ''
    extra = ', '.join(f'{name!r}' for name in names)
    cls_name = repr(self.__class__.__name__)
    raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

def _asdict(self) -> dict[str, Any]:
    return {
        name: getattr(self, name)
        for name, attr in self.__class__.__dict__.items()
        if isinstance(attr, Field)
    }

def __repr__(self) -> str:
    kwargs = ', '.join(
        f'{key}={value!r}' for key, value in self._asdict().items()
    )
    return f'{self.__class__.__name__}({kwargs})'

In [61]:
# Note: you would have a class decorator as well as __init_subclass__
print('@ builderlib module start')
class Builder:
    print('@ Builder body')

    def __init_subclass__(cls):
        print(f'@Builder.__init_subclass__({cls!r})')

        def inner_0(self):
            print(f'@ SuperA.__init_subclass__:inner_0({self!r})')

        cls.method_a = inner_0

    def __init__(self):
        super().__init__()
        print(f'@ Builder.__init__({self!r})')

def deco(cls):
    print(f'@ deco({cls!r})')

    def inner_1(self):
        print(f'@ deco:inner_1({self!r})')

    cls.method_b = inner_1
    return cls

class Descriptor:
    print('@ Descriptor body')

    def __init__(self):
        print(f'@ Descriptor.__init__({self!r})')

    def __set_name__(self, owner, name):
        args = (self, owner, name)
        print(f'@ Descriptor.__set_name__{args!r}')

    def __set__(self, instance, value):
        args = (self, instance, value)
        print(f'@ Descriptor.__set__{args!r}')

    def __repr__(self):
        return '<Descriptor instance>'

print('@ builderlib module end')

@ builderlib module start
@ Builder body
@ Descriptor body
@ builderlib module end


In [62]:
print('# evaldemo module start')

@deco
class Klass(Builder): # trigger  __init_subclass__
    print('# Klass body')

    attr = Descriptor()

    def __init__(self):
        super().__init__()
        print(f'# Klass.__init__({self!r})')

    def __repr__(self):
        return '<Klass instance>'

def main():
    obj = Klass()
    obj.method_a()
    obj.method_b()
    obj.attr = 999

if __name__ == '__main__':
    main()

print('# evaldemo module end')

# evaldemo module start
# Klass body
@ Descriptor.__init__(<Descriptor instance>)
@ Descriptor.__set_name__(<Descriptor instance>, <class '__main__.Klass'>, 'attr')
@Builder.__init_subclass__(<class '__main__.Klass'>)
@ deco(<class '__main__.Klass'>)
@ Builder.__init__(<Klass instance>)
# Klass.__init__(<Klass instance>)
@ SuperA.__init_subclass__:inner_0(<Klass instance>)
@ deco:inner_1(<Klass instance>)
@ Descriptor.__set__(<Descriptor instance>, <Klass instance>, 999)
# evaldemo module end


In [63]:
from collections.abc import Iterable
Iterable.__class__

abc.ABCMeta

In [64]:
import abc
from abc import ABCMeta
ABCMeta.__class__

type

In [66]:
# Every class is an instance of type, directly or indirectly. Type is an instance of itself!

In [68]:
# A metaclass inherits type
class MetaBunch(type):
    def __new__(meta_cls, cls_name, bases, cls_dict):
        defaults = {}
        def __init__(self, **kwargs):
            for name, default in defaults.items():
                setattr(self, name, kwargs.pop(name, default))
            if kwargs:
                extra = ', '.join(kwargs)
                raise AttributeError(f'No slots left for: {extra!r}')

        def __repr(self):
            rep = ', '.join(f'{name}={value!r}'
                            for name, default in defaults.items()
                            if (value := getattr(self, name)) != default)
            return f'{cls_name}({rep})'

        new_dict = dict(__slots__=[], __init__=__init__, __repr__=__repr__)

        for name, value in cls_dict.items():
            if name.startswith('__') and name.endswith('__'):
                if name in new_dict:
                    raise AttributeError(f"Can't set {name!r} in {cls_name!r}")
                new_dict[name] = value

            else:
                new_dict['__slots__'].append(name)
                defaults[name] = value
        return super().__new__(meta_cls, cls_name, bases, new_dict)

class Bunch(metaclass=MetaBunch):
    pass

In [74]:
import collections

class NosyDict(collections.UserDict):
    def __setitem__(self, key, value):
        args = (self, key, value)
        print(f'% NosyDict.__setitem__{args!r}')
        super().__setitem__(key, value)

    def __repr__(self):
        return '<Nosy instance>'

class MetaKlass(type):
    print('% MetaKlass body')

    @classmethod # gets called first
    def __prepare__(meta_cls, cls_name, bases):
        args = (meta_cls, cls_name, bases)
        print(f'% MetaKlass.__prepare__{args!r}')
        return NosyDict()

    def __new__(meta_cls, cls_name, bases, cls_dict):
        args = (meta_cls, cls_name, bases, cls_dict)
        print(f'% MetaKlass.__new__{args!r}')
        def inner_2(self):
            print(f'% MetaKlass.__new__:inner_2({self!r})')

        cls = super().__new__(meta_cls, cls_name, bases, cls_dict.data)
        cls.method_c = inner_2
        return cls

    def __repr__(cls):
        cls_name = cls.__name__
        return f'<class {cls_name!r} built by MetaKlass>'

print('% metalib module end')

% MetaKlass body
% metalib module end


In [76]:
@deco
class Klass(Builder, metaclass=MetaKlass):
    print('# Klass body')
    
    attr = Descriptor()

    def __init__(self):
        super().__init__()
        print(f'# Klass.__init__({self!r})')

    def __repr__(self):
        return '<Klass instance>'

def main():
    obj = Klass()
    # uncomment these to see how these get called
    # obj.method_a()
    # obj.method_b()
    # obj.method_c()
    # obj.attr = 420

if __name__ == '__main__':
    main()

print('# evaldemo_meta module end')

% MetaKlass.__prepare__(<class '__main__.MetaKlass'>, 'Klass', (<class '__main__.Builder'>,))
% NosyDict.__setitem__(<Nosy instance>, '__module__', '__main__')
% NosyDict.__setitem__(<Nosy instance>, '__qualname__', 'Klass')
# Klass body
@ Descriptor.__init__(<Descriptor instance>)
% NosyDict.__setitem__(<Nosy instance>, 'attr', <Descriptor instance>)
% NosyDict.__setitem__(<Nosy instance>, '__init__', <function Klass.__init__ at 0x106c6ff40>)
% NosyDict.__setitem__(<Nosy instance>, '__repr__', <function Klass.__repr__ at 0x106c6fd00>)
% NosyDict.__setitem__(<Nosy instance>, '__classcell__', <cell at 0x106c83100: empty>)
% MetaKlass.__new__(<class '__main__.MetaKlass'>, 'Klass', (<class '__main__.Builder'>,), <Nosy instance>)
@ Descriptor.__set_name__(<Descriptor instance>, <class 'Klass' built by MetaKlass>, 'attr')
@Builder.__init_subclass__(<class 'Klass' built by MetaKlass>)
@ deco(<class 'Klass' built by MetaKlass>)
@ Builder.__init__(<Klass instance>)
# Klass.__init__(<Klass inst

In [77]:
class Field:
    def __init__(self, name: str, constructor: Callable) -> None:
        if not callable(constructor) or constructor is type(None):
            raise TypeError(f'{name!r} type hint must be callable')
        self.name = name
        self.storage_name = '_' + name
        self.constructor = constructor

    def __get__(self, instance, owner=None):
        if instance is None:
            return self
        return getattr(instance, self.storage_name)

    def __set__(self, instance: Any, value: Any) -> None:
        if value is ...: # ... remember this determines if the value is None vs values that were not given
            value = self.constructor()
        else:
            try:
                value = self.constructor(value)
            except (TypeError, ValueError) as e:
                type_name = self.constructor.__name__
                msg = f'{value!r} is not compatible with {self.name}:{type_name}'
                raise TypeError(msg) from e
        setattr(instance, self.storage_name, value)
            

In [80]:
class CheckedMeta(type):
    def __new__(meta_cls, cls_name, bases, cls_dict):
        if '__slots__' not in cls_dict:
            slots = []
            type_hints = cls_dict.get('__annotations__', {}) # class does not exist yet, need to retrieve __annotations__
            for name, constructor in type_hints.items():
                field = Field(name, constructor)
                cls_dict[name] = field
                slots.append(field.storage_name)
            cls_dict['__slots__'] = slots
        return super().__new__(meta_cls, cls_name, bases, cls_dict)

In [81]:
class Checked(metaclass=CheckedMeta):
    __slots__ = () # skip CheckedMeta.__new__

    @classmethod
    def _fields(cls) -> dict[str, type]:
        return get_type_hints(cls)

    def __init__(self, **kwargs: Any) -> None:
        for name in self._fields():
            value = kwargs.pop(name, ...)
            setattr(self, name, value)
        if kwargs:
            set.__flag_unknown_attr(*kwargs)

    def __flag_unknwon_attrs(self, *names: str) -> NoReturn:
        plural = 's' if len(names) > 1 else ''
        extra = ', '.join(f'{name!r}' for name in names)
        cls_name = repr(self.__class__.__name__)
        raise AttributeError(f'{cls_name} object has no attribute{plural} {extra}')

    def _asdict(self) -> dict[str, Any]:
        return {
            name: getattr(self, name)
            for name, attr in self.__class__.__dict__.items()
            if isinstance(attr, Field)
        }

    def __repr__(self) -> str:
        kwargs = ', '.join(f'{key}={value!r}' for key, value in self._asdict().items())
        return f'{self.__class__.__name__}({kwargs})'

In [82]:
"""
Common use cases to use a metaclass are sometimes redundant because of new lanugage features:
    Class decorators
    __set_name__
    __init_subclass__
    built-in dict preserving insertion order

* I have a hunch that __slots__ (if not already) would be handled automatically by CPython compiler
so we wouldn't have to worry about manually doing this optimization. *
"""

"\nCommon use cases to use a metaclass are sometimes redundant because of new lanugage features:\n    Class decorators\n    __set_name__\n    __init_subclass__\n    built-in dict preserving insertion order\n\n* I have a hunch that __slots__ (if not already) would be handled automatically by CPython compiler\nso we wouldn't have to worry about manually doing this optimization. *\n"

In [88]:
# Another example of a MetaClass
# Autogenerates numeric constants
"""
End goal:
class Flavor(AutoConst):
    vanilla
    pistachio
    cheesecake

Flavor.vanilla
0
Flavor.pistachio, Flavor.cheesecake
(1, 2)
"""

class MissingConst(dict):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__next_value = 0

    def __missing__(self, key):
        if key.startswith('__') and key.endswith('__'):
            raise KeyError(key)
        self[key] = value = self.__next_value
        self.__next_value += 1
        return value

class AutoConstMeta(type):
    def __prepare__(name, bases, **kwargs):
        return MissingConst()

class AutoConst(metaclass=AutoConstMeta):
    pass

In [89]:
class Flavor(AutoConst):
    vanilla
    pistachio
    cheesecake

Flavor.vanilla

0

In [90]:
Flavor.pistachio, Flavor.cheesecake

(1, 2)